# Benchmark Repeat OPF

This notebook aims to benchmark the repeative optimal power flow.

In [1]:
import warnings
import logging
import ams
import numpy as np
import pandas as pd

import ams.benchmarks as bp

# Configure AMS logger
ams.config_logger(stream_level=20)

# Display tool versions
_ = bp.get_tool_versions()

ams.config_logger(stream_level=50)

Last run time: 2024-11-09 23:34:27
Python: 3.10.0 | packaged by conda-forge | (default, Nov 20 2021, 02:27:15) [Clang 11.1.0 ]

Tool        Version                      
----------  -----------------------------
ltbams      0.9.10.post153.dev0+gc73b7f8a
andes       1.9.1.post100+g2b17faac      
cvxpy       1.5.3                        
gurobipy    11.0.3                       
mosek       10.2.6                       
piqp        0.4.2                        
pandapower  2.14.11                      
numba       0.60.0                       


In [2]:
%matplotlib inline

In [3]:
cases_lfs = [
    ams.get_case('5bus/pjm5bus_uced.xlsx'),
    ams.get_case('ieee14/ieee14_uced.xlsx'),
    ams.get_case('ieee39/ieee39_uced.xlsx'),
    ams.get_case('npcc/npcc_uced.xlsx'),
    ams.get_case('wecc/wecc_uced.xlsx'),
]

lfs = [ams.load(case, setup=True).EDTSlot.sd.v[:, 0] for case in cases_lfs]

# store loading factor data
lfs_data = pd.DataFrame(data=np.array(lfs).T,
                        columns=['pglib_opf_case5_pjm', 'case14',
                                 'case39', 'npcc', 'wecc'])
lfs_data.to_csv('lfs_data.csv', index=False)

In [4]:
# Suppress warnings
warnings.filterwarnings('ignore')
# show only errors
logger = logging.getLogger('pandapower').setLevel(logging.ERROR)

cases = [
    './cases/case5.m',
    './cases/case14.m',
    './cases/case39.m',
    './cases/npcc.m',
    './cases/wecc.m',
]

case_names = [case.split('/')[-1].split('.')[0] for case in cases]

# Initialize data structures for storing results
n_iters = 10
n_cases = len(cases)

solvers = ['GUROBI', 'MOSEK', 'PIQP', 'pandapower']

time_data = np.zeros((n_iters, n_cases, len(bp.cols_pre) + len(solvers)))
obj_data = np.zeros((n_cases, len(solvers)))

In [5]:
for n_case, (case, lf) in enumerate(zip(cases, lfs)):
    print(f'Case: {case}')
    system = ams.load(case, setup=True, no_output=True, default_config=True)

    for n_iter in range(n_iters):
        pre_time, sol = bp.time_dcopf_with_lf(system,
                                              solvers=solvers,
                                              load_factors=lf,
                                              ignore_dpp=False)
        time_data[n_iter, n_case, :len(pre_time)] = np.array(
            [i for i in pre_time.values()])
        time_data[n_iter, n_case, len(pre_time):] = np.array(
            [sol[solver]['time'] for solver in solvers])

    obj = np.array([sol[solver]['obj'] for solver in solvers])
    obj_data[n_case, :] = obj

Case: ./cases/case5.m
Set parameter Username
Academic license - for non-commercial use only - expires 2025-09-25
Case: ./cases/case14.m
Case: ./cases/case39.m
Case: ./cases/npcc.m
Case: ./cases/wecc.m


In [6]:
# NOTE: results structure:
# 1st column: iteration
# 2nd column: cases
# 3rd column: solvers

obj_dcopf = pd.DataFrame(obj_data, columns=solvers, index=case_names)

time_dcopf = pd.DataFrame(columns=bp.cols_pre+solvers, index=case_names)
for case in case_names:
    time_dcopf.loc[case] = time_data[:, case_names.index(case), :].mean(axis=0)

time_dcopf.iloc[:, :] *= 1000        # scale to ms

In [7]:
time_dcopf

,ams_mats,ams_parse,ams_eval,ams_final,ams_postinit,GUROBI,MOSEK,PIQP,pandapower
case5,0.6,1.25,2.17,0.1,0.01,47.8,47.12,36.16,403.03
case14,0.67,1.21,2.14,0.09,0.01,54.92,53.85,41.17,280.8
case39,0.73,1.15,2.28,0.09,0.01,70.04,67.3,44.99,281.45
npcc,1.49,1.19,4.91,0.1,0.01,197.15,183.22,92.37,1246.63
wecc,1.61,1.22,3.43,0.1,0.01,196.25,154.21,96.59,884.5


In [8]:
obj_dcopf.round(2)

,GUROBI,MOSEK,PIQP,pandapower
case5,308818.76,308818.76,308818.76,308818.76
case14,136767.75,136767.75,136767.75,136767.75
case39,643403.27,643403.26,643403.27,643403.27
npcc,16635796.96,16635796.96,16635796.96,16634401.65
wecc,8621568.97,8621568.97,8621568.97,8621568.97


In [9]:
obj_dcopf.to_csv('../results/results_obj_repeat.csv')
time_dcopf.to_csv('../results/results_time_repeat.csv')